In [39]:
import numpy as np
import pandas as pd
# read the csv files
df = pd.read_csv('../data/rodpump_failure_final.csv')

In [40]:
# renaming the columns to make them more intuitive and easier to use
df.rename(columns={'roduid':'rod_uid',
                       'UWI':'uwi',
                       'NODEID':'well_name',
                       'IDWELL':'well_id',
                       'tbguid':'bha_tubing_id',
                       'lifetime_start':'bha_lifetime_start',
                       'lifetime_end' :'bha_lifetime_end',
                       'IDRECJOBPULL' : 'bha_lifetime_id',
                       'REPORTTO' : 'failure_type',
                       'H2S_CONCENTRATION' : 'h2s_conc',
                       'PrimarySetpoint' : 'primary_setpoint',
                       'SecondarySetpoint' : 'secondary_setpoint',
                       'StrokeLength' : 'stroke_len',
                       'GrossStrokeLength' : 'gross_stroke_len',
                       'Fillage' : 'fillage',
                       'YesterdaysAverageSPM' : 'yesterday_avg_spm',
                       'bha_configuration' : 'bha_config',
                       'gas_anchor_length' : 'gas_anchor_len',
                       'MAX_INCLINATION' : 'max_incline',
                       'AVG_PRESS_FLOWLINE' : 'avg_press_flowline',
                       'AVG_PRESSURE_TUBING' : 'avg_press_tubing',
                       'AVG_PRESSURE_CASING' : 'avg_press_casing',
                       'AVG_DIFFERENTIAL_PRESSURE' : 'avg_diff_press',
                       'AVG_OIL_VOLUME' : 'avg_oil_vol',
                       'AVG_WATER_VOLUME' : 'avg_water_vol',
                       'AVG_LIQUID_VOLUME' : 'avg_liquid_vol',
                       'AVG_WATERSG' : 'avg_watersg',
                       'ROUTE' : 'route',
                       'DESANDDEGAS_TYP' : 'dsand_dgas_type',
                       'CHROME_LENGTH' : 'chrome_len',
                       'ENDURALLOY_LENGTH' : 'enduralloy_len',
                       'POLY_LENGTH' : 'poly_len',
                       'NIPPLE_SET_DEPTH' : 'nip_set_depth',
                       'gasanchor_od' : 'gas_anchor_od'}, inplace = True)
# drop unnecessary columns
df.drop(columns = ['FAILSTART', 'FAILURETYPE'], inplace = True)

In [41]:
df['manual_scale'] = df['manual_scale'].map({'Y': True, 'N': False})
for col in df.columns:
    if len(df[col].unique()) == 2:
        df[col] = df[col].astype(bool)

# convert date columns to datetime and create a new column 'bha_lifetime'
date_cols = ['bha_lifetime_start', 'bha_lifetime_end']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

df['bha_lifetime'] = (df['bha_lifetime_end'] - df['bha_lifetime_start']).dt.days

In [42]:
# handling null values

# for numeric columns, fill missing values with the median
# use the median because the data is heavily skewed
for col in df.select_dtypes(include = 'number').columns:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

# for categorical columns, fill missing values with the mode (or "Unknown" if mode is not available)
for col in df.select_dtypes(include='object').columns:
    if col not in ["failure_type", "rod_uid", "well_name",
                   "well_id", "bha_tubing_id", "bha_lifetime_start",
                   "bha_lifetime_end", "bha_lifetime_id"]:  # Exclude failure_type and id columns
        if not df[col].mode().empty:
            mode_val = df[col].mode()[0]
        else:
            mode_val = "Unknown"
        df[col] = df[col].fillna(mode_val)
    elif col == "failure_type":
        df.loc[df["bha_lifetime_end"] == "2020-04-15 17:37:11.338", col] = df[col].fillna("No fail")
        df.loc[df["bha_lifetime_end"] != "2020-04-15 17:37:11.338", col] = df[col].fillna(df[col].mode()[0])


In [43]:
# outlier detection and removal (selective)

# Define the helper function to remove outliers using the IQR method.
def remove_outliers_iqr(data, col, multiplier=5.0):
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr
    return data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

# Define the list of columns for which we want to remove outliers.
# We are skipping 'stroke_len', 'yesterday_avg_spm', 'avg_press_flowline', and 'gas_anchor_len'
# because they appear to have valid variation even if their statistical spread is high.

cols_to_clean = [
    'fillage',
    'max_unguided_dls',
    'dls_high_in_hole',
    'max_incline',
    'avg_press_tubing',
    'avg_press_casing',
    'avg_diff_press',
    'avg_oil_vol',
    'avg_water_vol',
    'avg_liquid_vol',
    'overall_max_sideload',
    'shallow_max_sideload',
    'max_unguided_sideload',
    'chrome_len',
    'poly_len',
    'nip_set_depth'
]

# Track the initial row count.
print("Initial shape:", df.shape)

# Apply outlier removal only for columns in cols_to_clean.
for col in cols_to_clean:
    if col in df.columns:
        before_shape = df.shape
        df = remove_outliers_iqr(df, col, multiplier=3.0)
        after_shape = df.shape
        print(f"Removed outliers in {col}: {before_shape[0] - after_shape[0]} rows dropped")

# Apply percentile-based capping for 'stroke_len', 'yesterday_avg_spm', 'avg_press_flowline', and 'gas_anchor_len'
special_cols = ['yesterday_avg_spm', 'gas_anchor_len']
for col in special_cols:
    lower_limit = df[col].quantile(0.01)
    upper_limit = df[col].quantile(0.99)
    df[col] = df[col].clip(lower=lower_limit, upper=upper_limit)
    print(f"Capped outliers in {col} at 1st and 99th percentiles")

# Adjust outliers for stroke_len and gross_stroke_len by capping at 99th percentile
for col in ['stroke_len']:
    lower_limit = df[col].quantile(0.20)
    upper_limit = df[col].quantile(0.80)
    df[col] = df[col].clip(lower=lower_limit, upper=upper_limit)
    print(f"Capped outliers in {col} at 15th and 85th percentiles")

# Adjust outliers for stroke_len and gross_stroke_len by capping at 99th percentile
for col in ['gross_stroke_len','avg_press_flowline']:
    lower_limit = df[col].quantile(0.10)
    upper_limit = df[col].quantile(0.90)
    df[col] = df[col].clip(lower=lower_limit, upper=upper_limit)
    print(f"Capped outliers in {col} at 10th and 90th percentiles")

print("Final shape after selective outlier removal:", df.shape) # basically lost 24%ish outliers

Initial shape: (2596, 54)
Removed outliers in fillage: 102 rows dropped
Removed outliers in max_unguided_dls: 81 rows dropped
Removed outliers in dls_high_in_hole: 5 rows dropped
Removed outliers in max_incline: 57 rows dropped
Removed outliers in avg_press_tubing: 54 rows dropped
Removed outliers in avg_press_casing: 43 rows dropped
Removed outliers in avg_diff_press: 35 rows dropped
Removed outliers in avg_oil_vol: 7 rows dropped
Removed outliers in avg_water_vol: 39 rows dropped
Removed outliers in avg_liquid_vol: 1 rows dropped
Removed outliers in overall_max_sideload: 83 rows dropped
Removed outliers in shallow_max_sideload: 8 rows dropped
Removed outliers in max_unguided_sideload: 50 rows dropped
Removed outliers in chrome_len: 20 rows dropped
Removed outliers in poly_len: 25 rows dropped
Removed outliers in nip_set_depth: 9 rows dropped
Capped outliers in yesterday_avg_spm at 1st and 99th percentiles
Capped outliers in gas_anchor_len at 1st and 99th percentiles
Capped outliers i

In [44]:
df = df[df['failure_type'] != 'Liner (Casing)']

In [45]:
# Save the Final Cleaned Data
df.to_csv('../data/rod_cleaned_no_transform.csv', index=False)
print('\nData cleaning complete. Cleaned data saved as "rod_cleaned_final.csv".')


Data cleaning complete. Cleaned data saved as "rod_cleaned_final.csv".
